In [1]:
import numpy as np
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.size

104000

In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df.dropna(inplace=True)

In [5]:
df.size

91425

In [7]:
## Independat and dependant features Features
X=df.drop('label',axis=1)
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [11]:
## Approaching the problem using LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## defining the vocabulary size
voc_size=5000

In [12]:
msg=X.copy()
msg['title'][1] # let's see the first news title

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
msg.reset_index(inplace=True)
msg

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
import nltk
import re 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
## let's apply stemming to the words
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(msg)):
    rev=re.sub('[^a-zA-Z]',' ',msg['title'][i])
    rev=rev.lower()
    rev=rev.split()
    rev=[ps.stem(word) for word in rev if not word in stopwords.words('english')]
    rev=' '.join(rev)
    corpus.append(rev)

In [17]:
corpus[1:10] # checking the first 10 records in my data

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [18]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep[1:10]

[[3909, 3535, 274, 1044, 4751, 2540, 4438],
 [1645, 162, 423, 3814],
 [4581, 840, 3352, 806, 379, 2329],
 [2676, 4751, 2617, 4969, 2820, 4611, 4751, 4992, 1076, 2285],
 [1538,
  4233,
  3878,
  2763,
  36,
  1015,
  2573,
  4340,
  1192,
  1177,
  1697,
  4157,
  815,
  1973,
  4438],
 [4527, 4457, 1947, 3959, 3780, 2413, 1130, 4915, 2683, 563, 4532],
 [2346, 527, 2604, 3624, 2200, 2563, 1015, 4067, 2683, 563, 4532],
 [572, 2144, 3042, 2049, 141, 4418, 945, 4959, 1015, 738],
 [222, 1948, 4852, 2111, 3030, 3665, 2968, 4018]]

## Embedding representation

In [19]:
sent_len=20
embed_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_len)
print(embed_docs)

[[   0    0    0 ...  438 1652 1530]
 [   0    0    0 ... 4751 2540 4438]
 [   0    0    0 ...  162  423 3814]
 ...
 [   0    0    0 ... 2683  563 4532]
 [   0    0    0 ...  593 3971 3877]
 [   0    0    0 ... 1966  138 2039]]


In [23]:
X_final=np.array(embed_docs)
y_final=np.array(y)

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [26]:
X_train.shape

(12250, 20)

In [27]:
## creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.7894 - loss: 0.4147 - val_accuracy: 0.9155 - val_loss: 0.1946
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9488 - loss: 0.1333 - val_accuracy: 0.9218 - val_loss: 0.2037
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9678 - loss: 0.0875 - val_accuracy: 0.9211 - val_loss: 0.2107
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9834 - loss: 0.0527 - val_accuracy: 0.9193 - val_loss: 0.2620
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9874 - loss: 0.0411 - val_accuracy: 0.9186 - val_loss: 0.2997
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9926 - loss: 0.0252 - val_accuracy: 0.9171 - val_loss: 0.3593
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9971 - loss: 0.0120 - val_accuracy: 0.9145 - val_loss: 0.4294
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9984 - loss: 0.0072 - val_accu

## Performance Metrics and Accuracy

In [29]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.5,1,0) ## AUC ROC curve to select probabilities

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [38]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(confusion_matrix(y_test,y_pred))
print('\n Accuracy:',accuracy_score(y_test,y_pred))
print('\n',classification_report(y_test,y_pred))

[[3128  291]
 [ 218 2398]]

 Accuracy: 0.9156586578293289

               precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.92      0.90      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.92      0.92      6035

